In [ ]:
import numpy as np
import iris
import matplotlib.pyplot as plt
import netCDF4 as nc
import tensorflow as tf
from netCDF4 import Dataset
from keras.models import Model, Sequential
from keras.layers import Input, Dense, LeakyReLU, Dropout
from keras.layers import LeakyReLU
from keras.optimizers import Adam
import xarray as xr
from keras.models import model_from_json
from keras import backend
import random

Import Training Data and initialise random seed

In [ ]:
files = ['2021_t42_std.nc', '2022_t42_std.nc']
all_data = xr.open_mfdataset(files)
# select data to train on which is the first year
data = all_data.sel(time = '2021')
data_from_isca = xr.open_dataset('era_land_t42.nc')

In [ ]:
np.random.seed(123)

Function to normalise data

In [ ]:
def normalise(datain):

    dataout=(datain-np.min(datain))/(np.max(datain)-np.min(datain))

    return dataout;

In [ ]:
# this function will rejig the features one at a time and I'll normalise and concatenate them separately
def rejigging(data, plotting = False, times = range(len(data.time.values))): # times is just for plotting
    # Plots of each of the times
    if plotting:
        for i in times:
            df = data.sel(time=str(data.time.values[i]))
            plt.pcolormesh(df)
            plt.title(f'{data.time.values[i]}')
            
    # Starting with t2m - Do the first step manually by selecting the first time:
    df = data.sel(time=str(data.time.values[0]))
    # Reshape to 1d vector
    var = np.reshape(df.values,(64*128, 1))
    # Loop through the remaining times and append to a single array
    for i in range(1, len(data.time.values)):
        df = data.sel(time=str(data.time.values[i]))
        #reshape into 1d vector
        var_i = np.reshape(df.values,(64*128, 1))
        #concatenate the different times
        var = np.append(var, var_i, axis = 0)
    # Store min and max
    min_var = np.min(var)
    max_var = np.max(var)
    
    return var, min_var, max_var

In [ ]:
features = ['u10', 'v10', 't2m', 'sdor', 'rh2'] 
# t2m_std is what we predict - not included here so it can be last in the order later

feature_min = {} # to store training means in a dictionary
feature_max = {} # to store training stds in a dictionary
normalised_features = {} # to store normalised features for training in a dictionary

for f in features:
    df = data[f]
    reshaped = rejigging(df) # reshapes

    # stores mean and std for each feature
    feature_min[f'{f}'] = reshaped[1]
    feature_max[f'{f}'] = reshaped[2]

    # normalises the feature
    normalised_features[f'{f}'] = normalise(reshaped[0])

df = data_from_isca.zsurf
# Reshape to 1d vector
zsurf = np.reshape(df.values,(64*128, 1))
# Store mean and std
min_zsurf = np.min(zsurf)
max_zsurf = np.max(zsurf)
# Add copies for the different time points to match other vars
zsurf_i = zsurf
for i in range(1, len(data.time.values)):
    zsurf = np.append(zsurf, zsurf_i, axis = 0)
# normalise
normalised_zsurf = normalise(zsurf)

# Add to the lists of features made for main dataset
# stores mean and std for each feature
feature_min['zsurf'] = min_zsurf
feature_max['zsurf'] = max_zsurf
normalised_features['zsurf'] = normalised_zsurf

# Adding t2m_std last:
f = 't2m_std'

df = data[f]
reshaped = rejigging(df) # reshapes df

# stores mean and std for each feature
feature_min[f'{f}'] = reshaped[1]
feature_max[f'{f}'] = reshaped[2]

# normalises the feature
normalised_features[f'{f}'] = normalise(reshaped[0])

#define list with new feature names
features = list(normalised_features.keys())

# append to one dataset
big_data = np.append(normalised_features[features[0]], normalised_features[features[1]], axis = 1) # join 0th and 1st feature into big data
for i in range(2, len(normalised_features)): # join remaining features
    big_data=np.append(big_data, normalised_features[features[i]], axis = 1)

Histogram of normalised t2m_std - very unbalanced dataset...

In [ ]:
plt.rcParams['font.size'] = 12          # Default text size
plt.rcParams['axes.titlesize'] = 16     # Title font size
plt.rcParams['axes.labelsize'] = 13     # X and Y axis labels font size
plt.rcParams['xtick.labelsize'] = 12    # X tick labels font size
plt.rcParams['ytick.labelsize'] = 12    # Y tick labels font size
plt.rcParams['legend.fontsize'] = 13    # Legend font size
plt.rcParams['figure.titlesize'] = 18   # Figure title font size

In [ ]:
plt.figure(figsize = (5, 5))
plt.hist(normalised_features['t2m_std'], bins = 10)
plt.ylabel('Number of Samples')
plt.xlabel('Normalised t2m_std')
#plt.savefig('/home/links/sr850/isca_results/NN/NN_Training_Plots/hist_raw_data_normalistedt2m_std.png',  bbox_inches='tight', pad_inches=0.1)

Store Feature Maxima and Minima for unnormalising

In [ ]:
feature_min

In [ ]:
feature_max

In [ ]:
# Oversampling

norm_t2m_std = big_data[:, -1] # selecting last feature which is t2m_std

maybe = []
keep = []
for i in range(len(norm_t2m_std)):         #maybe I should add one for above 0.9...
    if  norm_t2m_std[i] >= .6:             # This criterion is to keep a proportion of indices of samples where std is greater than 0.8
        maybe.append(i)
    else:
        keep.append(i)

n= 3000 # just explicitly specify number n to keep from each of the bands

keep = keep + random.choices(maybe, k = n) # keep less of the higher values

resampled_big_data = np.zeros((len(keep), np.shape(big_data)[1])) 
for j in range(np.shape(big_data)[1]): # loops through 0 to 7 for the 8 features
    for w in range(len(keep)): # cycles through the indices you've selected to keep
        resampled_big_data[w, j] = big_data[keep[w], j] # assigns data to keep from big_data to resampled_big_data
        
plt.figure(figsize = (5, 5))
plt.hist(resampled_big_data[:, 6], bins = 10)
plt.ylabel('Number of Samples')
plt.xlabel('Normalised t2m_std')
#plt.savefig('/home/links/sr850/isca_results/NN/NN_Training_Plots/hist_resampled_data_normalistedt2m_std.png',  bbox_inches='tight', pad_inches=0.1)

In [ ]:
plt.figure(figsize = (5, 5))
plt.hist(resampled_big_data[:, 6], bins = 10)
plt.ylabel('Number of Samples')
plt.xlabel('Normalised t2m_std')
#plt.savefig('/home/links/sr850/isca_results/NN/NN_Training_Plots/hist_resampled_data_normalistedt2m_std.png',  bbox_inches='tight', pad_inches=0.1)

Reshape prediction to match Isca's Grid and Unnormalise and perform validation

In [ ]:
np.random.seed(123)
# Shuffle

#shuffled_big_data   = np.random.permutation(big_data) # Changed to the resampled dataset
shuffled_big_data   = np.random.permutation(resampled_big_data) # Changed to the resampled dataset

# 7 input features 
n_inputs=6 

# So having put all the data together to shuffle, we now separate it again into its inputs/features/x
# and the outputs/target/y.

x     = shuffled_big_data[:,        :n_inputs]  # all rows, columns 0 up to n_inputs - 1 (so here we get the 0th and 1st column)
y     = shuffled_big_data[:,n_inputs:        ]

# The the first n % of the data and use that for training, the remainder will be used for validation.

nt            = x.shape[0]

n = 0.8 # 80% for training

n_train       = int(n*nt)

trainx, testx = x[:n_train, :], x[n_train:, :]

trainy, testy = y[:n_train], y[n_train:]

# START: THINGS YOU MAY WISH TO VARY
# TIER 1: (more worthwhile looking at the impact of changin these)
# Define the number of nodes in each layer

n_nodes = 16

# The total number of layers (yes you could have different number of nodes in each layer, but this is simpler).
n_layers = 4
epochs   = 20

# TIER 2: (you could change these too, but maybe try the ones above first).
# Define the leakiness of the ReLU
leaky_alpha = 0.1
# Define likelihood of a node being randomly ignored during training.
drop_out_rate = 0.2

initial_learning_rate = 1.0e-3
batch_size            = 128

# Later we use a learning rate scheduler than reduces the learning rate every n epochs.
reduce_every_n        = 5
# END: THINGS YOU MAY WISH TO VARY
# Start of definition of the multi-layer perceptron (MLP) or fully-connected neural network.
# Note how after x=Dense()(input), the syntax is that x=f(x)
# This means that it adds things to x progressively.
# Here n_input is 4, for avg_theta, avg_orog, std_orog and avg_lsm
input = Input(shape=(n_inputs,))
X    = Dense(n_nodes)(input)
X     = LeakyReLU(alpha=leaky_alpha)(X)

# These are the hidden layers, you could define several layers manually, or use a nice loop to do it for yo.
for layer in np.arange(1,n_layers):
    X     = Dense(n_nodes)(X)
    X     = Dropout(drop_out_rate)(X)
    X     = LeakyReLU(alpha=leaky_alpha)(X)

# The final or output layer, this has dimension 1, for the single prediction of std_theta that we are trying to predict.

X     = Dense(1, activation='relu')(X)

# Define the whole model is terms of the inputs and the things accumulated in x

model = Model(inputs=input, outputs=X)

# Prints everything to screen.
# Do have a look at the output shape at each stage and the number of parameters associated with each layer.
# Also look at final "total number of parameters".
# Confirm that you are happy how these numbers are related to the number of nodes and layers you have defined.
model.summary()

# Learning rate scheduler
# reduce the learning rate (i.e. the size of the updates to the weights and biases)
# to help you home in on thebest value.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=int(nt / batch_size)*reduce_every_n,
                                                             decay_rate=0.5,staircase=True)

# Use the Adam optmizer.
opt=tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# State the loss that you want to minimise, here mean squared error.
model.compile(optimizer=opt, loss='mse', metrics=['mse', 'R2Score'])
#model.compile(optimizer=opt, loss=my_mse_loss, metrics=['mse'])
#model.compile(optimizer=opt, loss=my_m2e_loss, metrics=['mse'])
#model.compile(optimizer=opt, loss=my_m4e_loss, metrics=['mse'])
#model.compile(optimizer=opt, loss='msle', metrics=['mse', 'R2Score'])

# Do the actual training.
# This learns to predict trainy using trainx
# and also calculates how well it does at predicting testy using testx, which was NOT used as part of the minimisations.
history=model.fit(trainx, trainy, validation_data=(testx, testy), epochs=epochs, batch_size=batch_size, shuffle=True)
# This cost function is calculated using batch_size number of samples at a time.
# An update is done after every batch. It will therefore take n_train/batch_size training steps to get through all the data once.
# This is called an "epoch". We then shuffle the data before doing the next epoch.

# Get hold of the mse for the training and validation (witheld) data.
# Ideally you want a loss MSE for both. But not a lower MSE for train than for val (that suggests over-fitting to the train data).
mse_train = history.history['loss']
mse_val   = history.history['val_loss']
R2Score_train = history.history['R2Score']
R2Score_val = history.history['val_R2Score']

# and plot it
epoch_vec=np.arange(1,epochs+1)
plt.figure()
plt.plot(epoch_vec,mse_train,'k-')
plt.plot(epoch_vec,mse_val,'b--')
plt.title('mse')
plt.show()
# this allows you to see whether things have plateaued off, or whether there is overfitting.

my_string='SR_resampled_minusd2m'+str(n_layers)+'L_'+str(n_nodes)+'N'
# Write this info out
fileout=my_string+'_mse_train.txt'
np.savetxt(fileout, np.ones((1,1))*mse_train, fmt='%10.7f')
fileout=my_string+'_mse_valid.txt'
np.savetxt(fileout, np.ones((1,1))*mse_val, fmt='%10.7f')
# Write the model architecture to one file.
model_json=model.to_json()
fileout=my_string+'.json'
with open(fileout, "w") as json_file:
    json_file.write(model_json)
    json_file.close()

# Write all the trained weights and biases to another file.
    fileout=my_string+'.weights.h5'
    model.save_weights(fileout)
    
print('All done!')

In [ ]:
plt.figure()
plt.plot(epoch_vec,mse_train,'g-', label = 'training data')
plt.plot(epoch_vec,mse_val,'b--', label = 'test data')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.xticks(np.arange(0, 21, 5))
plt.ylim(0.0045, 0.0095)
plt.legend()
plt.savefig(f'/home/links/sr850/isca_results/NN/NN_Training_Plots/mse_trainingcurve_resampled_mse_20ep_{str(n_nodes)+'_' + str(n_layers)}.png',  bbox_inches='tight', pad_inches=0.1)

In [ ]:
predy = model.predict(testx)
plt.figure()
plt.plot(testy,predy,'k+')
plt.plot([0,1],[0,1],'r-')
plt.xlabel('Normalised truth ')
plt.ylabel('Normalised prediction')
plt.title('Test Data')
plt.show() 

## Now Define and normalise validation dataset

In [ ]:
validation_data = all_data.sel(time = '2022')

In [ ]:
def normalise_with_training(datain, training_min , training_max):
    dataout=(datain-training_min)/(training_max-training_min)
    return dataout;

In [ ]:
# this function will rejig the features WITHOUT NORMALISING
def rejigging2(data, plotting = False, times = range(len(data.time.values))): # times is just for plotting
    # Plots of each of the times
    if plotting:
        for i in times:
            df = data.sel(time=str(data.time.values[i]))
            plt.pcolormesh(df)
            plt.title(f'{data.time.values[i]}')
            
    # Starting with t2m - Do the first step manually by selecting the first time:
    df = data.sel(time=str(data.time.values[0]))
    # Reshape to 1d vector
    var = np.reshape(df.values,(64*128, 1))
    # Loop through the remaining times and append to a single array
    for i in range(1, len(data.time.values)):
        df = data.sel(time=str(data.time.values[i]))
        #reshape into 1d vector
        var_i = np.reshape(df.values,(64*128, 1))
        #concatenate the different times
        var = np.append(var, var_i, axis = 0)
    # Store min and max
    min_var = np.min(var)
    max_var = np.max(var)

    return var, min_var, max_var

Select validation data (ideally for two months because the plotting is set up to plot the first of two):

In [ ]:
unseen_data = validation_data.sel(time = slice(validation_data.time.values[0],validation_data.time.values[1]))

In [ ]:
features = ['u10', 'v10', 't2m', 'sdor', 'rh2'] 
unseen_normalised_features = {}

for f in features:
    df = unseen_data[f]
    reshaped = rejigging2(df)
    df = reshaped[0] # reshapes df

    # normalises the feature
    unseen_normalised_features[f'{f}'] = normalise_with_training(df, training_min = feature_min[f'{f}'], training_max = feature_max[f'{f}'])

df = data_from_isca.zsurf
# Reshape to 1d vector
zsurf = np.reshape(df.values,(64*128, 1))
# Add copies for the different time points to match other vars
zsurf_i = zsurf
for i in range(1, len(unseen_data.time.values)):
    zsurf = np.append(zsurf, zsurf_i, axis = 0)
# normalise
unseen_normalised_features['zsurf'] = normalise_with_training(zsurf, training_min = feature_min['zsurf'], training_max = feature_max['zsurf'])

# Adding t2m_std last:
f = 't2m_std'

df = unseen_data[f]
reshaped = rejigging2(df)
df = reshaped[0] # reshapes df

# normalises the feature
unseen_normalised_features[f'{f}'] = normalise_with_training(df, training_min = feature_min[f'{f}'], training_max = feature_max[f'{f}'])

features = list(unseen_normalised_features.keys())

unseen_big_data = np.append(unseen_normalised_features[features[0]], unseen_normalised_features[features[1]], axis = 1) # join 0th and 1st feature into big data
for i in range(2, len(unseen_normalised_features)): # join remaining features
    unseen_big_data=np.append(unseen_big_data, unseen_normalised_features[features[i]], axis = 1)

Prediction Step

In [ ]:
unseen_x     = unseen_big_data[:,        :n_inputs]  # all rows, columns 0 up to n_inputs - 1
unseen_y     = unseen_big_data[:,n_inputs:        ]
pred_unseen_y = model.predict(unseen_x)

In [ ]:
plt.figure()
plt.plot(unseen_y,pred_unseen_y,'k+')
plt.plot([0,1],[0,1],'r-')
plt.xlabel('Normalised truth (unseen data)')
plt.ylabel('Normalised prediction (unseen data)')
plt.title('Not Seen In Training')
plt.show()

In [ ]:
def reshape_to_2d(data_at_specific_time): # this assumes you run validation for two times
    t = data_at_specific_time[0:int(16384/2),:] # 16384 must be np.shape(pred_unseen_y)[0], since you run validation for two times
    reshaped = np.reshape(t, (64, 128))
    return reshaped

In [ ]:
reshaped = reshape_to_2d(pred_unseen_y)

In [ ]:
def unnormalise(normalised_data, training_min , training_max):
    return ((training_max - training_min) *(normalised_data)) + training_min

In [ ]:
def truth_plot(ax): # this needs unseen_data
    t2m_std_t0 = unseen_data.t2m_std.sel(time = unseen_data.time.values[0], method = 'nearest')
    contours = ax.contourf(unseen_data.lon, unseen_data.lat, t2m_std_t0, levels = np.arange(0, 11, 1), extend = 'max')
    plt.colorbar(contours)
    ax.set_title('Truth')
    return t2m_std_t0 ;
def prediction_plot(ax): # this needs pred_unseen_y and feature_min and max t2m_std
    unnormalised_prediction = unnormalise(pred_unseen_y, feature_min['t2m_std'], feature_max['t2m_std'])
    reshaped = reshape_to_2d(unnormalised_prediction)
    # this is normalised
    contours = ax.contourf(unseen_data.lon, unseen_data.lat, reshaped, levels = np.arange(0, 11, 1), extend = 'max')
    plt.colorbar(contours)
    ax.set_title('Prediction')
    return reshaped;
def prediction_minus_truth(ax,  difference): # this also needs unseen_data to be defined
    l = int(np.abs((difference)).values.max()) # takes largest absolute value from difference and rounds to integer to set cbar lims
    print('colorbar lim should be', l)
    l=5 # trying to pick one to standardise across them all, but printing just to check I haven't set it too low
    diff = ax.contourf(unseen_data.lon, unseen_data.lat, difference, levels = np.arange(-l, l+.01, 0.1), extend = 'both', cmap = 'seismic')
    plt.colorbar(diff)
    ax.set_title('Prediction - Truth')
    return diff ;
def unnormalised_truth_vs_prediction(ax, y , y_pred):
    ax.plot(y,y_pred,'k+')
    ax.plot([0, 10],[0,10],'r-')
    ax.set_xlabel('Truth')
    ax.set_ylabel('Prediction')
    ax.set_title('Truth VS Prediction (unseen data)')
    return ;

Validation Figure

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 4, figsize = (16, 3))
truth = truth_plot(axs[0])
prediction = prediction_plot(axs[1])
prediction_minus_truth(axs[2], prediction - truth)
unnormalised_truth_vs_prediction(axs[3], y = np.reshape(truth,(64*128, 1)), y_pred = unnormalise(pred_unseen_y, feature_min['t2m_std'], feature_max['t2m_std'])[0:64*128])
for i in [0, 1, 2]:
    axs[i].set_xlabel('Latitude')
    axs[i].set_ylabel('Longitude')
fig.tight_layout()

plt.savefig(f'/home/links/sr850/isca_results/NN/NN_Training_Plots/validation_plot_resampled_mse_{str(n_nodes)+'_' + str(n_layers)}.png',  bbox_inches='tight', pad_inches=0.1)

In [ ]:
model.evaluate(unseen_x, unseen_y)